In [ ]:
!pip install langchain
!pip install langchain_community
!pip install langgraph
!pip install pandas
!pip install langchain_experimental
!pip install ibm_watson_machine_learning
!pip install langchain-ibm

In [11]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from langchain_ibm import WatsonxEmbeddings, WatsonxLLM

In [ ]:
llm= WatsonxLLM(
    model_id="ibm/granite-3-8b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    apikey="xxxxxxxxxxxxxx__4XBZxxxxxxxxxxxxxL",
    project_id="xxxxxxxxxxxxxxxxx_bb1b-exxxxxxxxxxxxx",
    params={
      GenParams.DECODING_METHOD: "greedy",
      GenParams.TEMPERATURE: 0.7,
      GenParams.MIN_NEW_TOKENS: 5,
      GenParams.MAX_NEW_TOKENS:1000,
    },
)

In [14]:
from typing import TypedDict, List, Dict, Optional
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import pandas as pd
import re
import json
from datetime import datetime
import asyncio

# Load and preprocess datasets
def load_datasets():
    datasets = {
        'ratings': pd.read_csv('Ratings&Sentiments_POC - Sheet1.csv'),
        'complaints': pd.read_csv('Complaints_POC - Sheet1.csv'),
        'sales': pd.read_csv('Sales_POC - Sheet1.csv'),
        'marketing': pd.read_csv('Marketing_POC - Sheet1.csv'),
        'teamperformance': pd.read_csv('TeamPerformance_POC - Sheet1.csv')
    }

    # Standardize column names and types
    for name, df in datasets.items():
        df.columns = df.columns.str.strip().str.lower().str.replace('[^a-z0-9_]', '_', regex=True)

        # Convert date columns
        for col in df.columns:
            if 'date' in col:
                df[col] = pd.to_datetime(df[col])

        # Convert numeric columns
        for col in df.select_dtypes(include=['object']).columns:
            try:
                df[col] = pd.to_numeric(df[col])
            except:
                pass

        # Standardize string values
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].astype(str).str.strip().str.upper()

    return datasets

datasets = load_datasets()

class GraphState(TypedDict):
    query: str
    interpretation: dict
    department_results: dict
    final_response: str
    error: Optional[str]

llm = llm

async def interpret_query(state: GraphState):
    """Convert natural language queries into structured data requests"""
    system_msg = """You are an expert query interpreter. Convert the user's query into JSON format specifying:

1. departments: Which datasets to query (ratings, complaints, sales, marketing, teamperformance)
2. products: List of product IDs (PROD001-PROD007) if specified
3. metrics: Specific columns to retrieve
4. aggregation: How to process the data (sum, avg, count, list)

Example outputs:
1. "Total revenue for PROD001":
{
  "departments": ["sales"],
  "products": ["PROD001"],
  "metrics": ["net_revenue"],
  "aggregation": "sum"
}

2. "Complaints about PROD006":
{
  "departments": ["complaints"],
  "products": ["PROD006"],
  "metrics": ["issue_description"],
  "aggregation": "list"
}"""

    try:
        response = await llm.ainvoke([
            SystemMessage(content=system_msg),
            HumanMessage(content=state["query"])
        ])
        interpretation = json.loads(response.content)

        # Validate the interpretation
        required_keys = ['departments', 'products', 'metrics', 'aggregation']
        if not all(k in interpretation for k in required_keys):
            raise ValueError("Missing required fields in interpretation")

        return {"interpretation": interpretation, "error": None}
    except Exception as e:
        return {"interpretation": {}, "error": f"Interpretation failed: {str(e)}"}

async def process_departments(state: GraphState):
    """Execute data processing based on the interpretation"""
    if not state.get("interpretation"):
        return {"department_results": {}, "error": "No interpretation provided"}

    results = {}
    interpretation = state["interpretation"]

    for dept in interpretation["departments"]:
        if dept not in datasets:
            continue

        df = datasets[dept]
        dept_results = {}

        # Handle product filtering
        products = interpretation["products"] or (
            df['product_id'].unique().tolist() if 'product_id' in df.columns else ['ALL']
        )

        for product in products:
            if 'product_id' in df.columns:
                product_data = df[df['product_id'].str.upper() == product.upper()]
            else:
                product_data = df

            if product_data.empty:
                continue

            metric_results = {}
            for metric in interpretation["metrics"]:
                if metric not in product_data.columns:
                    continue

                # Apply the requested aggregation
                agg_func = interpretation["aggregation"]
                if agg_func == "sum":
                    value = product_data[metric].sum()
                elif agg_func in ["avg", "mean"]:
                    value = round(product_data[metric].mean(), 2)
                elif agg_func == "count":
                    value = product_data[metric].count()
                elif agg_func == "list":
                    value = product_data[metric].tolist()
                else:  # Default to first value
                    value = product_data[metric].iloc[0]

                metric_results[metric] = value

            if metric_results:
                dept_results[product] = metric_results

        if dept_results:
            results[dept] = dept_results

    return {"department_results": results, "error": None}

async def generate_response(state: GraphState):
    """Generate comprehensive business report with insights and recommendations"""
    if not state.get("department_results"):
        return await fallback_response(state["query"])

    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a senior business analyst. Create a detailed report with these sections:

1. KEY FINDINGS:
   - Present all requested metrics clearly with proper formatting
   - Include comparisons where relevant
   - Highlight key statistics

2. KEY INSIGHTS:
   - Analyze what the numbers mean in business context
   - Identify patterns, trends, and anomalies
   - Explain potential causes for observed results

3. RECOMMENDATIONS:
   - Provide 3-5 actionable business recommendations
   - Prioritize by potential impact
   - Include implementation considerations

4. CONCLUSION:
   - Summary of key takeaways
   - Next steps for the business
   - Final thoughts

Data: {data}

Current Date: {current_date}"""),
        ("human", "Original Query: {query}")
    ])

    try:
        response = await (prompt | llm).ainvoke({
            "query": state["query"],
            "data": json.dumps(state["department_results"], indent=2),
            "current_date": datetime.now().strftime("%Y-%m-%d")
        })
        return {"final_response": response.content, "error": None}
    except Exception as e:
        return await fallback_response(state["query"])

async def fallback_response(query: str):
    """Direct data lookup with insights when interpretation fails"""
    results = {}

    # Try to extract product IDs
    products = re.findall(r'PROD\d{3}', query.upper()) or ['ALL']

    # Check all datasets for relevant data
    for name, df in datasets.items():
        dept_results = {}

        for product in products:
            if 'product_id' in df.columns:
                product_data = df[df['product_id'].str.upper() == product.upper()]
            else:
                product_data = df

            if product_data.empty:
                continue

            # Auto-detect metrics based on query
            if 'rating' in query.lower() and 'rating' in df.columns:
                dept_results[product] = {
                    'average_rating': round(product_data['rating'].mean(), 2),
                    'rating_distribution': product_data['rating'].value_counts().to_dict()
                }
            elif 'unit' in query.lower() and 'units_sold' in df.columns:
                dept_results[product] = {
                    'total_units_sold': int(product_data['units_sold'].sum())
                }
            elif 'revenue' in query.lower() and 'net_revenue' in df.columns:
                dept_results[product] = {
                    'total_revenue': product_data['net_revenue'].sum()
                }
            elif 'complaint' in query.lower() and 'issue_description' in df.columns:
                dept_results[product] = {
                    'complaints': product_data['issue_description'].tolist(),
                    'recurring_issues': product_data[product_data['recurring_flag'] == 'YES']['issue_description'].tolist()
                }

        if dept_results:
            results[name] = dept_results

    if results:
        # Generate comprehensive insights from the direct data
        insight_prompt = ChatPromptTemplate.from_messages([
            ("system", """Generate a detailed business report with:

1. KEY FINDINGS: Present the data clearly
2. INSIGHTS: Analyze patterns and business implications
3. RECOMMENDATIONS: 3-5 actionable suggestions
4. CONCLUSION: Summary and next steps

Data: {data}"""),
            ("human", "Query: {query}")
        ])

        insight_response = await (insight_prompt | llm).ainvoke({
            "query": query,
            "data": json.dumps(results, indent=2)
        })

        return {
            "final_response": insight_response.content,
            "error": None
        }

    return {"final_response": "No relevant data found. Please try rephrasing your query.", "error": None}

# Build the workflow graph
workflow = StateGraph(GraphState)
workflow.add_node("interpret", interpret_query)
workflow.add_node("process", process_departments)
workflow.add_node("respond", generate_response)
workflow.add_edge("interpret", "process")
workflow.add_edge("process", "respond")
workflow.add_edge("respond", END)
workflow.set_entry_point("interpret")
app = workflow.compile()

async def execute_query(query: str):
    """Run the full analysis pipeline"""
    result = await app.ainvoke({"query": query})
    return result.get("final_response", "No response generated")

# Example test cases
test_queries = [
    "Total revenue generated for PROD001 and PROD002"
]

async def run_demo():
    print("="*60)
    print("BUSINESS ANALYTICS REPORTING SYSTEM")
    print("="*60)

    for query in test_queries:
        print(f"\nQUERY: {query}")
        print("-"*60)
        response = await execute_query(query)
        print(response)
        print("-"*60)

# Proper async context handling
if __name__ == "__main__":
    try:
        asyncio.run(run_demo())
    except RuntimeError as e:
        if "running event loop" in str(e):
            loop = asyncio.get_event_loop()
            loop.run_until_complete(run_demo())
        else:
            raise

BUSINESS ANALYTICS REPORTING SYSTEM

QUERY: Total revenue generated for PROD001 and PROD002
------------------------------------------------------------
### 1. KEY FINDINGS:

- **Total Revenue Generated:**
  - **PROD001:** $25,384.32
  - **PROD002:** $18,604.30
  - **Combined Total Revenue:** $43,988.62

- **Comparison:**
  - PROD001 generated 36.55% more revenue than PROD002.

### 2. KEY INSIGHTS:

- **Revenue Distribution:**
  - PROD001 contributes significantly more to the total revenue compared to PROD002. This indicates a higher market demand or better market penetration for PROD001.

- **Market Performance:**
  - The difference in revenue between the two products suggests varying levels of consumer preference or possibly different pricing strategies. PROD001’s higher revenue could be indicative of higher pricing, greater volume sold, or a combination of both.

- **Potential Market Trends:**
  - The substantial revenue from PROD001 might reflect a trend where customers prefer its 